### Task: Simplest Read All

In [4]:
import os
os.chdir(r'D:\OneDrive\programs\pstsgkit\tests')

In [5]:
from sgkit_plink._open_bed import open_bed

val = open_bed('datasets/snpgen.bed').read(force_python_only=True)
print(val.shape, val.dtype)
val

(1000, 5) int8


array([[   2, -127,    2,    2,    2],
       [   2,    2,    2,    2,    2],
       [-127,    2, -127,    2, -127],
       ...,
       [-127, -127,    2,    2, -127],
       [   2,    2,    2,    2, -127],
       [   2,    2,    2,    2,    2]], dtype=int8)

#### Discussion

* This is running code at https://github.com/CarlKCarlK/sgkit-plink/commit/b1ab9e04
* In this example, it avoids parsing the metadata (samples and variant names, etc)
   * All metadata parsing is now lazy
   * In this case, it does a one-time, fast, line count of the \*.fam and \*.bim files instead of a parse.
* "close()" is not needed because there is no longer a persistent file pointer

*In the future*

* *force_python_only* won't be needed
* Final import will be something like 'from sgkit_plink import open_bed

### Task: sgkit's use case: Reading data in batches when we know the dimensions

In [6]:
import numpy as np

with open_bed('datasets/all_chr.maf0.001.N300.bed',iid=300,sid=1015) as bed:
    batch_size = 100
    for start in range(0,bed.sid_count,batch_size):
        val = bed.read(index=np.s_[:,start:start+batch_size],force_python_only=True)
        print(val.shape)

(300, 100)
(300, 100)
(300, 100)
(300, 100)
(300, 100)
(300, 100)
(300, 100)
(300, 100)
(300, 100)
(300, 100)
(300, 15)


#### Discussion

* This is sgkit's use case
* The \*.fam and \*.bim files are never read.
* I'm thinking about adding an \_\_item\_\_ method
   * But PySnpTools needs to specify a dtype and order on every read

### Task: Find the mean value for every chromosome (excluding missing values) 

In [7]:
with open_bed('datasets/all_chr.maf0.001.N300.bed') as bed:
    for chrom in np.unique(bed.chromosome): # pos[:,0] is chrom number
        val = bed.read(index=np.s_[:,bed.chromosome==chrom],dtype='float32',force_python_only=True)
        print(f'chromo {int(chrom)} mean: {np.nanmean(val):0.4}')

chromo 1 mean: 0.1631
chromo 10 mean: 0.2113
chromo 11 mean: 0.2175
chromo 12 mean: 0.208
chromo 13 mean: 0.1667
chromo 14 mean: 0.2361
chromo 15 mean: 0.1573
chromo 16 mean: 0.1735
chromo 17 mean: 0.1155
chromo 18 mean: 0.1494
chromo 19 mean: 0.234
chromo 2 mean: 0.1015
chromo 20 mean: 0.1828
chromo 21 mean: 0.126
chromo 22 mean: 0.2173
chromo 23 mean: 0.3713
chromo 3 mean: 0.1927
chromo 4 mean: 0.318
chromo 5 mean: 0.2038
chromo 6 mean: 0.1897
chromo 7 mean: 0.185
chromo 8 mean: 0.191
chromo 9 mean: 0.204


#### Discussion

* This examples shows what a NumPy-inspired API is good at.
* Lazy metadata means, the '.bim' file is parsed (one time) here while the '.fam' file is merely line-counted (one time).

### Punt Slicing Metadata?

* My plan is to continue to ignore the rest of the metadata info (e.g., parent info, phenotype in \*.fam) because:
  * I've never had anyone request it
  * They can parse it from \*.fam and \*.bim if they need it
* However, I can imagine a way to efficently slice into metadata, but is this ability needed?

In [ ]:
#hypothetical! doesn't run. may not implement

with open_bed('datasets/all_chr.maf0.001.N300.bed') as bed:
    batch_size = 100
    
    # The first time .sid_count is called, C++ code would (in one pass)
    # count the lines in *.bim and return the offsets for sqrt(#lines) random lines.
    # For example, if there turns out to be 1 million variants, 
    # we'd remember offsets to a random 1000 of them.
    
    for start in range(0,bed.sid_count,batch_size):
        #'batch' becomes an open_bed object for just a subset of the data
        batch = bed[:,start:start+batch_size]
        print(batch.sid) #We can find all the variant names in batch in Order(sqrt(#original_lines))
        val = batch.read()
        print(val.shape)
        
# Could do for *.fam and *.bim

#### Discussion

* This doesn't require a cache file
* It is low memory and fast
* But ...
  * Is it needed by anyone?
  * If it is needed, can't they just use sgkit and its dask files?

### random stuff

In [10]:
%%time
filename = r'M:\deldir\genbgen\2\merged_487400x220000.1.bed'
bed = open_bed(filename)
bed.shape

Wall time: 3.12 s


(487400, 220000)

In [13]:
bed.read(np.s_[:,100*1000:100*1000+10])

array([[   2, -127,    2, ...,    2,    2,    2],
       [   2, -127,    2, ...,    2, -127,    2],
       [   2, -127,    2, ...,    2,    2,    2],
       ...,
       [   2,    2,    2, ...,    2,    2,    2],
       [   2,    2, -127, ...,    2, -127,    2],
       [   2, -127,    2, ...,    2,    2, -127]], dtype=int8)

In [20]:
bed.chromosome

array(['1', '1', '1', ..., '1', '1', '1'], dtype='<U1')

In [21]:
bed.iid

array([['1', 'sample_0'],
       ['2', 'sample_1'],
       ['3', 'sample_2'],
       ...,
       ['487398', 'sample_48739'],
       ['487399', 'sample_48739'],
       ['487400', 'sample_48739']], dtype='<U12')

In [33]:
sum(range(1000*1000))

499999500000

In [49]:
from pysnptools.util.mapreduce1 import map_reduce
from pysnptools.util.mapreduce1.runner import LocalMultiThread, LocalMultiProc, Local
import time

def slow_square(x):
    time.sleep(1)
    return x*x

def square_sum(count,runner=None):
    ss= map_reduce(range(count),
               mapper=slow_square,
               reducer=sum,
               runner=runner)
    return ss

square_sum(10)

285

In [50]:
%%time

square_sum(10,runner=Local())

Wall time: 10.1 s


285

In [52]:
%%time

square_sum(100,runner=LocalMultiThread(10))

Wall time: 10.1 s


328350

In [62]:
def thread_read(filename, runner):
    with open_bed(filename) as bed:
        bed.shape #causes the lazy meta to read the # lines in the two metadata files
        batch_size = 100
        def read_and_report(start):
            print(start)
            val = bed.read(index=np.s_[:,start:start+batch_size],force_python_only=True)
            return val.shape

        report = map_reduce(range(0,bed.sid_count,batch_size),
                   mapper=read_and_report,
                   runner=runner)
        return report



In [63]:
%%time

thread_read('datasets/all_chr.maf0.001.N300.bed', runner=Local())

0
100
200
300
400
500
600
700
800
900
1000
Wall time: 67 ms


[(300, 100),
 (300, 100),
 (300, 100),
 (300, 100),
 (300, 100),
 (300, 100),
 (300, 100),
 (300, 100),
 (300, 100),
 (300, 100),
 (300, 15)]

In [64]:
%%time

thread_read('datasets/all_chr.maf0.001.N300.bed', runner=LocalMultiThread(10))

0100

200
300
400
500
600
700
800
900
1000
Wall time: 93 ms


[(300, 100),
 (300, 100),
 (300, 100),
 (300, 100),
 (300, 100),
 (300, 100),
 (300, 100),
 (300, 100),
 (300, 100),
 (300, 100),
 (300, 15)]

In [61]:
bigfile = r'M:\deldir\genbgen\2\merged_487400x220000.1.bed'

In [65]:
%%time

thread_read(bigfile, runner=Local())

0
100
200
300
400
500
600
700


KeyboardInterrupt: 

In [ ]:
%%time

thread_read(bigfile, runner=LocalMultiThread(10))

0
22000
44000
66000
88000
110000
132000
154000
176000
198000
661008810044100

22100176100


154100198100
100
110100
132100

20044200

8820066200

132200
110200176200
198200154200


22200
198300
88300110300

132300
66300154300
17630022300

300
44300

198400
11040088400
154400

66400400
17640044400
22400


132400
198500
176500
110500
1545004450022500

6650013250088500
500



198600
6660088600600
110600176600

132600

44600
22600

154600
198700
8870066700

44700
176700700

154700110700

13270022700

198800
13280066800
154800
800
88800110800

17680044800

22800

198900
15490066900

8890022900132900

110900

176900
19900044900900


670002300089000


45000
177000155000
1000

133000199100111000


67100
199200
89100
15510045100

177100
133100
23100
1111001100

67200
199300
89200
177200155200

2320045200

133200
1200111200

67300
199400
89300
23300
177300
155300
45300
133300
1300
111300
67400
199500
89400
177400
2340045400

155400
111400
1400133400

67500
199600
89500
155500
23500177500

111500

79600101400

211700
12500
1885001444001225005650034400




166600
79700101500

211800
12600
16670056600188600144500
34500


122600

79800101600

211900
34600
1270056700
122700144600


188700166800

79900101700

212000
12800122800
188800144700

5680016690034700



80000
101800
212100
122900
144800
1889001290034800
167000


56900
80100
101900
212200
144900189000123000

13000167100


57000
34900
10200080200

212300
1310057100

102100
8030035000
189100123100

167200

145000
212400
167300145100189200


5720035100102200123200
80400


13200

212500
13300
102300
5730080500
21260014520016740035200
189300




123300
80600
212700
35300189400
10240013400

167500

145300
123400
57400
80700
212800
102500
13500
123500
1895001676003540057500145400




80800
212900
102600
5760035500145500
13600
167700

189600

123600
80900
213000
102700
189700
167800
577003560013700


145600123700

81000
213100
102800
189800123800

167900
3570013800
145700

57800
21320081100

102900
12390018990035800

145800
1680001390